In [1]:
################## IMPORT LIBRARIES ##################

import numpy as np
import random 
import pandas as pd
import os
from os.path import join as pjoin

In [2]:
################## IMPORT MY MODULES ##################

import sys
sys.path.append('../src')
import helpers as hlp
import importlib
importlib.reload(hlp);

In [18]:
np.random.seed(42)

# datapath="/media/ssd2/RESULTS-reverb-match-cond-u-net/"
datapath="/home/ubuntu/Data/"

In [19]:
# ---------- SPEECH POOL ------------

# create df with paths to speech files
speech_dataset_path1 =  pjoin(datapath,'VCTK','wav48_silence_trimmed')
speech_dataset_path2 =  pjoin(datapath,'PTDB')

# initialize empty list of files:
speech_pool = []

# fill the list of files with filenames from vctk data base:
database="VCTK"
for root, dirs, files in os.walk(speech_dataset_path1):
    for file in files:
        if file.endswith('.flac'):
            # decide which split based on a probability 
            speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})

# fill the list of files with filenames from ptdb data base:
database="PTDB"
for root, dirs, files in os.walk(speech_dataset_path2):
    for file in files:
        # make sure the correct speech files are used (MIC directory)
        if ("/MIC/" in root) & (file.endswith('.wav')):
            # decide which split based on a probability 
            speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})  

# shuffle order
random.shuffle(speech_pool)

# turn list to data frame 
speech_pool = pd.DataFrame(speech_pool)
print(f"{len(speech_pool)=}")


len(speech_pool)=93046


In [22]:
# ---------- RIR POOL ------------
rir_path=pjoin(datapath,'synth_rirs_mono')

database="synth_rirs_mono"
# load df with rirs paths and stats (it was generated together with the RIRs - rir_dataset.ipynb):
rir_pool=pd.read_csv(pjoin(rir_path,"rir_info.csv"),index_col=0)
rir_pool["database_rir"]=database
print(f"{len(rir_pool)=}")

len(rir_pool)=10000


In [23]:
# --------- CREATE METADATA FOR A DATASET BEING COMBINATIONS OF SPEECH AND RIRS --------
from datetime import datetime
date_tag = datetime.now().strftime("%d-%m-%Y--%H-%M")
# create dataset with 150000 data points, which consists of random combinations of speech, noise and rirs
N_datapoints=150000

# # sample from noise pool: 
# df_noise=noise_pool.sample(N_datapoints,replace=True)
# # here plan ways to augment noise data set:
# random_bool_values = [random.choice([1, -1]) for _ in range(len(df_noise))]
# df_noise["aug_phase"]=random_bool_values

# sample from speech pool: 
df_speech=speech_pool.sample(N_datapoints,replace=True)
# here plan ways to augment noise data set:
random_bool_values = [random.choice([1, -1]) for _ in range(len(df_speech))]
df_speech["aug_phase"]=random_bool_values

# sample from rir pool: 
df_rir=rir_pool.sample(N_datapoints,replace=True)

# concatenate samples from speech, noise and rir pools
# df_ds = pd.concat([df_speech.reset_index(drop=True), df_noise.reset_index(drop=True), df_rir.reset_index(drop=True)], axis=1,ignore_index=False)
df_ds = pd.concat([df_speech.reset_index(drop=True), df_rir.reset_index(drop=True)], axis=1,ignore_index=False)

df_ds = df_ds.reset_index(drop=True)

# randomize snr (only high snrs)
# df_ds["snr"]= 200 #np.random.uniform(low=10, high=30, size=len(df_ds))

# Create test-train-val split:
df_ds.loc[0:N_datapoints*0.8,"split"]="train" # 80% training data
df_ds.loc[N_datapoints*0.8:N_datapoints*0.9,"split"]="test" # 10% testing data
df_ds.loc[N_datapoints*0.9:N_datapoints,"split"]="val" # 10% validation data

# save dataset metadata:
# df_ds.to_csv(date_tag+"_data_set.csv")


In [24]:
# # Change paths in the metadata from GuestXR computer: 

# df = pd.read_csv("../dataset-metadata/nonoise2_guestxr2.csv",index_col=0)

# df["speech_file_path"]=df["speech_file_path"].str.replace("/home/ubuntu/Data/","/media/ssd2/")
# df["noise_file_path"]=df["noise_file_path"].str.replace("/home/ubuntu/Data/","/media/ssd2/")
# df["ir_file_path"]=df["ir_file_path"].str.replace("/home/ubuntu/Data/","/media/ssd2/")

# df.to_csv("../dataset-metadata/nonoise2_dacom.csv",index=False)

# df.head(10)

In [ ]:
# Check generate database

df = pd.read_csv("../dataset-metadata/17-05-2024--15-42_data_set.csv",index_col=0)
